In [376]:
import math
import sklearn
import numpy

import theano
import theano.tensor as T
import lasagne

import json

import pandas as pd
from sklearn.cross_validation import train_test_split

import lasagne.regularization as reg

In [377]:
good = open("../scripts/codestyle_stats/fine_res.json", "r")
bad = open("../scripts/codestyle_stats/shit_res.json", "r")

In [378]:
bad_data = bad.read().replace('\n', ' ')
bad_dict = json.loads(bad_data)

In [379]:
good_data = good.read().replace('\n', ' ')
good_dict = json.loads(good_data)

In [380]:
print len(bad_dict)

608


In [381]:
print len(good_dict)
print [0]*75

2540
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [382]:
bad = []
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in bad_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(bad_dict[i]['errors'].values())
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in bad_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(int(value))
    #print data
    s =numpy.array(s) 
    bad.append(s)
bad = numpy.array(bad)
print bad.shape
# for i in range(0, len(X_train)):
#     for elem in X_train[i]:
#         if not (isinstance( elem, int )):
#             print i
#             print type(elem)
#print X_train

(608, 82)


In [383]:
print bad.shape

(608, 82)


In [384]:
good = []
for i in range(0, len(bad_dict)):
    s = []
    if ('errors' in good_dict[i]):
        #print bad_dict[i]['errors']
        s.extend(numpy.array(good_dict[i]['errors'].values()))
        #print s
    else:
        s.extend([0]*(len(bad_dict[0]['errors'])))
    #print s
    #print s
    for key, value in good_dict[i].items():
         if key not in ('errors', 'filename'):
            s.append(int(value))
            #print data
    s = numpy.array(s)
    good.append(s)
good = numpy.array(good)
print good.shape
print good[0]
    #print s

(608, 82)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  0 17  0  0]


In [385]:
X_train = []
X_train.extend(good)
X_train.extend(bad)
X_train = numpy.array(X_train)

X_train.shape

(1216, 82)

In [386]:
y_train = numpy.array([1]*len(bad_dict) + [0]* len(bad_dict), dtype=numpy.int32)

In [387]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4)
for s in (X_train, X_val, y_train, y_val):
    print(s.shape)

(729, 82)
(487, 82)
(729,)
(487,)


In [388]:
print numpy.shape(y_train)

(729,)


In [389]:
print numpy.shape(X_train)

(729, 82)


In [390]:
def build_network(input_var=None):
    l_in = lasagne.layers.InputLayer(shape=(None, 82),
                                     input_var=input_var)
    l_hid = lasagne.layers.DenseLayer(l_in, num_units=82, 
                                       nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_hid2 = lasagne.layers.DenseLayer(l_hid, num_units=41, 
                                       nonlinearity=lasagne.nonlinearities.rectify,
                                       W=lasagne.init.GlorotUniform())
    l_hid3 = lasagne.layers.DenseLayer(l_hid2, num_units=20, 
                                       nonlinearity=lasagne.nonlinearities.softmax,
                                       W=lasagne.init.GlorotUniform())
    l_out = lasagne.layers.DenseLayer(l_hid3, num_units=2, 
                                      nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_out

In [391]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    #print inputs
    #print targets
    if shuffle:
        indices = numpy.arange(len(inputs))
        numpy.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [392]:
input_var = T.matrix('inputs')
target_var = T.ivector('targets')

In [393]:
network = build_network(input_var)

In [394]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

In [395]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params,
                                            learning_rate=0.01,
                                            momentum=0.9)
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(
                                                    test_prediction,
                                                    target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

In [396]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [397]:
for epoch in range(700):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        for batch in iterate_minibatches(X_train, y_train, 50, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 50, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        # Then we print the results for this epoch:
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

  training loss:		0.691700
  validation loss:		0.689306
  validation accuracy:		46.89 %
  training loss:		0.714238
  validation loss:		0.750160
  validation accuracy:		46.89 %
  training loss:		0.709289
  validation loss:		0.701812
  validation accuracy:		42.67 %
  training loss:		0.696875
  validation loss:		0.695194
  validation accuracy:		53.11 %
  training loss:		0.696305
  validation loss:		0.695363
  validation accuracy:		41.11 %
  training loss:		0.695220
  validation loss:		0.696867
  validation accuracy:		43.78 %
  training loss:		0.694738
  validation loss:		0.697007
  validation accuracy:		45.78 %
  training loss:		0.694700
  validation loss:		0.694902
  validation accuracy:		43.78 %
  training loss:		0.693692
  validation loss:		0.696415
  validation accuracy:		46.67 %
  training loss:		0.694240
  validation loss:		0.695727
  validation accuracy:		46.67 %
  training loss:		0.693773
  validation loss:		0.695416
  validation accuracy:		46.67 %
  training loss:		0.693244
  val

  training loss:		0.590295
  validation loss:		0.592860
  validation accuracy:		70.22 %
  training loss:		0.575355
  validation loss:		0.624923
  validation accuracy:		64.22 %
  training loss:		0.607364
  validation loss:		0.596385
  validation accuracy:		66.89 %
  training loss:		0.639832
  validation loss:		0.614003
  validation accuracy:		67.78 %
  training loss:		0.592123
  validation loss:		0.608849
  validation accuracy:		68.67 %
  training loss:		0.633722
  validation loss:		0.665427
  validation accuracy:		62.44 %
  training loss:		0.711425
  validation loss:		0.646021
  validation accuracy:		63.33 %
  training loss:		0.634156
  validation loss:		0.645030
  validation accuracy:		67.56 %
  training loss:		0.662286
  validation loss:		0.653508
  validation accuracy:		62.67 %
  training loss:		0.663327
  validation loss:		0.653492
  validation accuracy:		65.56 %
  training loss:		0.651433
  validation loss:		0.662454
  validation accuracy:		61.56 %
  training loss:		0.632916
  val

  training loss:		0.586177
  validation loss:		0.613882
  validation accuracy:		66.89 %
  training loss:		0.585129
  validation loss:		0.592794
  validation accuracy:		68.44 %
  training loss:		0.602040
  validation loss:		0.605974
  validation accuracy:		66.44 %
  training loss:		0.590277
  validation loss:		0.590638
  validation accuracy:		70.22 %
  training loss:		0.580933
  validation loss:		0.591277
  validation accuracy:		69.78 %
  training loss:		0.574366
  validation loss:		0.572413
  validation accuracy:		70.89 %
  training loss:		0.573047
  validation loss:		0.592439
  validation accuracy:		70.22 %
  training loss:		0.611821
  validation loss:		0.611959
  validation accuracy:		68.00 %
  training loss:		0.568233
  validation loss:		0.588536
  validation accuracy:		69.11 %
  training loss:		0.580802
  validation loss:		0.672473
  validation accuracy:		59.33 %
  training loss:		0.596560
  validation loss:		0.583098
  validation accuracy:		70.44 %
  training loss:		0.580859
  val

  training loss:		0.535660
  validation loss:		0.573599
  validation accuracy:		72.44 %
  training loss:		0.565892
  validation loss:		0.551732
  validation accuracy:		74.22 %
  training loss:		0.520200
  validation loss:		0.547381
  validation accuracy:		74.44 %
  training loss:		0.524348
  validation loss:		0.585630
  validation accuracy:		70.44 %
  training loss:		0.550061
  validation loss:		0.604659
  validation accuracy:		68.22 %
  training loss:		0.562558
  validation loss:		0.566960
  validation accuracy:		71.11 %
  training loss:		0.543437
  validation loss:		0.564468
  validation accuracy:		70.89 %
  training loss:		0.525721
  validation loss:		0.624245
  validation accuracy:		65.78 %
  training loss:		0.544508
  validation loss:		0.545767
  validation accuracy:		74.44 %
  training loss:		0.535006
  validation loss:		0.544879
  validation accuracy:		74.44 %
  training loss:		0.536573
  validation loss:		0.574836
  validation accuracy:		72.22 %
  training loss:		0.540496
  val

  training loss:		0.545553
  validation loss:		0.538892
  validation accuracy:		75.11 %
  training loss:		0.522231
  validation loss:		0.537929
  validation accuracy:		74.67 %
  training loss:		0.524353
  validation loss:		0.567337
  validation accuracy:		70.89 %
  training loss:		0.525840
  validation loss:		0.541824
  validation accuracy:		72.89 %
  training loss:		0.517403
  validation loss:		0.540833
  validation accuracy:		75.11 %
  training loss:		0.519358
  validation loss:		0.540111
  validation accuracy:		75.11 %
  training loss:		0.525465
  validation loss:		0.587387
  validation accuracy:		70.89 %
  training loss:		0.531613
  validation loss:		0.540459
  validation accuracy:		73.78 %
  training loss:		0.523517
  validation loss:		0.569274
  validation accuracy:		71.56 %
  training loss:		0.524151
  validation loss:		0.570281
  validation accuracy:		71.33 %
  training loss:		0.522145
  validation loss:		0.554735
  validation accuracy:		72.22 %
  training loss:		0.514587
  val

  training loss:		0.559612
  validation loss:		0.552108
  validation accuracy:		74.22 %
  training loss:		0.522428
  validation loss:		0.576282
  validation accuracy:		72.00 %
  training loss:		0.555183
  validation loss:		0.539708
  validation accuracy:		75.78 %
  training loss:		0.520305
  validation loss:		0.544836
  validation accuracy:		74.00 %
  training loss:		0.562574
  validation loss:		0.570519
  validation accuracy:		73.11 %
  training loss:		0.518324
  validation loss:		0.694125
  validation accuracy:		61.78 %
  training loss:		0.634621
  validation loss:		0.535822
  validation accuracy:		76.22 %
  training loss:		0.544407
  validation loss:		0.533606
  validation accuracy:		75.56 %
  training loss:		0.514968
  validation loss:		0.537518
  validation accuracy:		72.89 %
  training loss:		0.529173
  validation loss:		0.544026
  validation accuracy:		72.00 %
  training loss:		0.511354
  validation loss:		0.534060
  validation accuracy:		75.33 %
  training loss:		0.549653
  val

  training loss:		0.493914
  validation loss:		0.538752
  validation accuracy:		74.00 %
  training loss:		0.498670
  validation loss:		0.517729
  validation accuracy:		74.67 %
  training loss:		0.522641
  validation loss:		0.551766
  validation accuracy:		72.44 %
  training loss:		0.493688
  validation loss:		0.517016
  validation accuracy:		76.22 %
  training loss:		0.517228
  validation loss:		0.516866
  validation accuracy:		75.56 %
  training loss:		0.514755
  validation loss:		0.525441
  validation accuracy:		74.44 %
  training loss:		0.506839
  validation loss:		0.524780
  validation accuracy:		74.67 %
  training loss:		0.498829
  validation loss:		0.515462
  validation accuracy:		76.22 %
  training loss:		0.499212
  validation loss:		0.524064
  validation accuracy:		73.33 %
  training loss:		0.485051
  validation loss:		0.514330
  validation accuracy:		77.33 %
  training loss:		0.506956
  validation loss:		0.515597
  validation accuracy:		75.78 %
  training loss:		0.495429
  val

  training loss:		0.503677
  validation loss:		0.510287
  validation accuracy:		76.89 %
  training loss:		0.485033
  validation loss:		0.510948
  validation accuracy:		76.89 %
  training loss:		0.500822
  validation loss:		0.551474
  validation accuracy:		73.11 %
  training loss:		0.494059
  validation loss:		0.536521
  validation accuracy:		72.89 %
  training loss:		0.527084
  validation loss:		0.518550
  validation accuracy:		76.00 %
  training loss:		0.495231
  validation loss:		0.508650
  validation accuracy:		77.56 %
  training loss:		0.491232
  validation loss:		0.515433
  validation accuracy:		74.44 %
  training loss:		0.483061
  validation loss:		0.509736
  validation accuracy:		76.89 %
  training loss:		0.498291
  validation loss:		0.518001
  validation accuracy:		74.67 %
  training loss:		0.500616
  validation loss:		0.538607
  validation accuracy:		73.33 %
  training loss:		0.500579
  validation loss:		0.513901
  validation accuracy:		76.44 %
  training loss:		0.485761
  val

In [401]:
def iterate_minibatches_2(inputs, batchsize):
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt]
        
        
# use trained network for predictions
test_prediction = lasagne.layers.get_output(network, deterministic=True)
predict_fn = theano.function([input_var], T.argmax(test_prediction, axis=1))

test_data = good
sum = 0
for batch in iterate_minibatches_2(inputs=test_data,batchsize=1):
        inputs = batch
        result=predict_fn(inputs)
        sum += result[0]
print len(good)
print sum

test_data = bad
sum = 0
for batch in iterate_minibatches_2(inputs=test_data,batchsize=1):
        inputs = batch
        result=predict_fn(inputs)
        sum += result[0]
print len(bad)
print len(bad)-sum
494+399

608
494
608
399


893